In [38]:
import numpy as np
import csv
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold
from scipy import interp
from  matplotlib import pyplot as plt
from sklearn import cross_validation
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
from sklearn import metrics, preprocessing
from sklearn.preprocessing import OneHotEncoder
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [159]:
data = pd.read_csv('train.csv')

In [160]:
data.drop(data.columns[[0, 3]], axis=1, inplace=True)

In [161]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35,1,0,113803,53.1000,C123,S
4,0,3,male,35,0,0,373450,8.0500,NaN,S


In [162]:
data['Sex'] = data['Sex'].map({'female': 1, 'male': 0})

In [163]:
data = data.fillna(data.mean())

In [164]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,0,22,1,0,A/5 21171,7.2500,NaN,S
1,1,1,1,38,1,0,PC 17599,71.2833,C85,C
2,1,3,1,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,1,35,1,0,113803,53.1000,C123,S
4,0,3,0,35,0,0,373450,8.0500,NaN,S


In [165]:
data = data.fillna(0)

In [166]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,0,22,1,0,A/5 21171,7.2500,0,S
1,1,1,1,38,1,0,PC 17599,71.2833,C85,C
2,1,3,1,26,0,0,STON/O2. 3101282,7.9250,0,S
3,1,1,1,35,1,0,113803,53.1000,C123,S
4,0,3,0,35,0,0,373450,8.0500,0,S


In [167]:
df_dummy = pd.get_dummies(data, prefix=['Ticket', 'Cabin', 'Embarked'])

In [169]:
df_dummy.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Ticket_110152,Ticket_110413,Ticket_110465,...,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_0,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22,1,0,7.2500,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,38,1,0,71.2833,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,3,1,26,0,0,7.9250,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,35,1,0,53.1000,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,3,0,35,0,0,8.0500,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [178]:
data = pd.merge(data, df_dummy)

In [179]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,...,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_0,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22,1,0,A/5 21171,7.2500,0,S,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,38,1,0,PC 17599,71.2833,C85,C,...,0,0,0,0,0,0,0,1,0,0
2,1,3,1,26,0,0,STON/O2. 3101282,7.9250,0,S,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,35,1,0,113803,53.1000,C123,S,...,0,0,0,0,0,0,0,0,0,1
4,0,3,0,35,0,0,373450,8.0500,0,S,...,0,0,0,0,0,0,0,0,0,1


In [183]:
data.drop(data.columns[[ 6, 8, 9]], axis=1, inplace=True)

//anaconda/lib/python3.4/site-packages/pandas/core/index.py:1087: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  result = getitem(key)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [184]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Ticket_110152,Ticket_110413,Ticket_110465,...,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_0,Embarked_C,Embarked_Q,Embarked_S
0,0,3,0,22,1,0,7.2500,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,1,38,1,0,71.2833,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,3,1,26,0,0,7.9250,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,35,1,0,53.1000,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,3,0,35,0,0,8.0500,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [121]:
data['Ticket_enc'] = pd.factorize(data['Ticket'])[0]
data['Cabin_enc'] = pd.factorize(data['Cabin'])[0]
data['Embarked_enc'] = pd.factorize(data['Embarked'])[0]

In [185]:
data.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Ticket_110152,Ticket_110413,Ticket_110465,...,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_0,Embarked_C,Embarked_Q,Embarked_S
count,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,...,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000
mean,0.266752,2.554563,0.253989,29.551434,0.414805,0.246331,22.806457,0.001914,0.001914,0.001276,...,0.001914,0.001914,0.005743,0.001276,0.002553,0.000638,0.001276,0.156350,0.131461,0.710913
std,0.442403,0.730246,0.435430,10.087300,1.208837,0.666147,39.713131,0.043727,0.043727,0.035714,...,0.043727,0.043727,0.075592,0.035714,0.050475,0.025262,0.035714,0.363303,0.338012,0.453483
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,28.000000,0.000000,0.000000,7.895800,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,3.000000,0.000000,29.699118,0.000000,0.000000,8.050000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,3.000000,1.000000,29.699118,0.000000,0.000000,21.852100,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [171]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,0,22,1,0,A/5 21171,7.2500,0,S
1,1,1,1,38,1,0,PC 17599,71.2833,C85,C
2,1,3,1,26,0,0,STON/O2. 3101282,7.9250,0,S
3,1,1,1,35,1,0,113803,53.1000,C123,S
4,0,3,0,35,0,0,373450,8.0500,0,S


In [123]:
data.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Ticket_enc,Cabin_enc,Embarked_enc
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,29.699118,0.523008,0.381594,32.204208,306.859708,15.527497,0.368126
std,0.486592,0.836071,0.477990,13.002015,1.102743,0.806057,49.693429,197.286045,34.996694,0.647601
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,135.000000,0.000000,0.000000
50%,0.000000,3.000000,0.000000,29.699118,0.000000,0.000000,14.454200,285.000000,0.000000,0.000000
75%,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,474.500000,0.000000,1.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,680.000000,147.000000,3.000000


In [124]:
data.drop(data.columns[[ 6, 8, 9, 10]], axis=1, inplace=True)

In [125]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin_enc,Embarked_enc
0,0,3,0,22,1,0,7.2500,0,0
1,1,1,1,38,1,0,71.2833,1,1
2,1,3,1,26,0,0,7.9250,0,0
3,1,1,1,35,1,0,53.1000,2,0
4,0,3,0,35,0,0,8.0500,0,0


In [126]:
data.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin_enc,Embarked_enc
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.352413,29.699118,0.523008,0.381594,32.204208,15.527497,0.368126
std,0.486592,0.836071,0.477990,13.002015,1.102743,0.806057,49.693429,34.996694,0.647601
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000
50%,0.000000,3.000000,0.000000,29.699118,0.000000,0.000000,14.454200,0.000000,0.000000
75%,1.000000,3.000000,1.000000,35.000000,1.000000,0.000000,31.000000,0.000000,1.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,147.000000,3.000000


In [158]:
df_dummy = pd.get_dummies(data, prefix=['Cabin_enc', 'Embarked_enc'])

ValueError: Length of 'prefix' (2) did not match the length of the columns being encoded (0).

In [157]:
df_dummy.head(5)

,Cabin_enc,Embarked_enc
0,0,0
1,1,1
2,0,0
3,2,0
4,0,0


In [147]:
cat_dict = data[['Cabin_enc', 'Embarked_enc']].T.to_dict().values()

In [110]:
from sklearn.feature_extraction import DictVectorizer

In [149]:
vectorizer = DictVectorizer()

In [150]:
vec_x_cat_train = vectorizer.fit_transform(cat_dict )

  (1, 0)	1.0
  (1, 1)	1.0
  (3, 0)	2.0


In [103]:
enc = OneHotEncoder()

In [143]:
encoded = enc.fit(data[["Cabin_enc","Embarked_enc"]]) 

In [146]:
print(encoded)

OneHotEncoder(categorical_features='all', dtype=<class 'float'>,
       handle_unknown='error', n_values='auto', sparse=True)


In [128]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin_enc,Embarked_enc
0,0,3,0,22,1,0,7.2500,0,0
1,1,1,1,38,1,0,71.2833,1,1
2,1,3,1,26,0,0,7.9250,0,0
3,1,1,1,35,1,0,53.1000,2,0
4,0,3,0,35,0,0,8.0500,0,0


In [77]:
data.drop(data.columns[[ 6]], axis=1, inplace=True)

In [186]:
y = data['Survived'].as_matrix()

In [187]:
X =data.iloc[:,1:].values

In [191]:
y.shape

(1567,)

In [192]:
np.any(np.isnan(X))


False

In [193]:
print(X[-1])

[  3.     0.    32.     0.     0.     7.75   0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.

In [194]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X, y, test_size=0.25, random_state=42)

In [195]:
pre = preprocessing.StandardScaler()

In [196]:
pre.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [203]:
clf = LogisticRegression(C=100)
clf.fit(pre.transform(X_train),y_train)
clf.score(pre.transform(X_test),y_test)

0.90561224489795922

In [198]:
scores = cross_validation.cross_val_score(clf, X, y, cv=5)

In [199]:
scores.mean()

0.89715860796170532

In [200]:
def evaluate(C):
    return LogisticRegression(C=C) \
        .fit(pre.fit_transform(X_train), y_train) \
        .score(pre.transform(X_test), y_test)

In [201]:
def bestModel(allCs):
    """ Iterates through all of the C values we pass, and returns a tuple of the score of the best C,
    and the best C"""
    score = 0
    c_val = .001
    for c in allCs:
        currScore = evaluate(c)
        if currScore > score:
            score = currScore
            c_val = c
    return score, c_val

In [202]:
possible_Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
bestC = bestModel(possible_Cs)
print(bestC)

(0.90561224489795922, 100)
